# Nuclio - prediction function

## Setup the environment

In [4]:
# nuclio: ignore
import nuclio

### Set environment variables

In [100]:
# Iguazio access
%nuclio env FRAMESD=${V3IO_FRAMESD}
%nuclio env V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}

# Model handling
%nuclio env MODEL_FILE=lgb.model
%nuclio env -c MODEL_FILEPATH=/tmp/mlmodel/${MODEL_FILE}
%nuclio env -l MODEL_FILEPATH=models/trained/${MODEL_FILE}

# Function variables
%nuclio env NUM_OF_PRODUCTS_TO_RETURN=4
%nuclio env CUSTOMERS_TABLE=customers

%nuclio: setting 'FRAMESD' environment variable
%nuclio: setting 'V3IO_USERNAME' environment variable
%nuclio: setting 'V3IO_ACCESS_KEY' environment variable
%nuclio: setting 'MODEL_FILE' environment variable
%nuclio: setting 'MODEL_FILEPATH' environment variable
%nuclio: setting 'NUM_OF_PRODUCTS_TO_RETURN' environment variable
%nuclio: setting 'CUSTOMERS_TABLE' environment variable


### Base image

In [301]:
%nuclio config spec.build.baseImage = "python:3.6-jessie"

%nuclio: setting spec.build.baseImage to 'python:3.6-jessie'


### Installations

In [302]:
%%nuclio cmd  
pip install requests
pip install pandas
pip install lightgbm
pip install v3io_frames

### Imports

In [303]:
# Util
import os
import json
import requests
import datetime

# Function
import pandas as pd
import lightgbm as lgb

# DB
import v3io_frames as v3f

## Function code

### Init context

In [304]:
def init_context(context):
    # define DB
    client = v3f.Client('http://' + os.environ['FRAMESD'])
    setattr(context, 'client', client)
    
    setattr(context, 'customers_table', os.environ['CUSTOMERS_TABLE'])
    
    # define Model
    model = lgb.Booster(model_file=os.environ['MODEL_FILEPATH'])
    setattr(context, 'model', model)
    
    # vars
    setattr(context, 'k', int(os.environ['NUM_OF_PRODUCTS_TO_RETURN']))

### Format dataframe for prediction

In [305]:
def prepare_df(df):
    # Extract features col
    keep_cols = ['products']
    df = df.loc[:, keep_cols]
    
    # Create features df
    df = pd.read_json(df.values[0][0])
    return df

In [306]:
def handler(context, event):

    # Get user parameters
    df = context.client.read('kv', context.customers_table, filter=f'id=={event.body["id"]}').reset_index()

    # Do we have features for the user?
    if not df.empty:
                             
        # Create features df for the user
        df = prepare_df(df)
                            
        # Predict
        df["prediction"] = context.model.predict(df)
                             
        # Eliminate low confidence results
        df = df.loc[df.prediction > 0.01, ['prediction', 'product_id']]

        # Keep top products
        df.sort_values(by='prediction', ascending=False, inplace=True)
        best_products = df.iloc[:context.k, :]


        return json.dumps(best_products.to_json())
    return json.dumps({})

In [307]:
# nuclio: ignore
init_context(context)

In [308]:
# nuclio: ignore
event = nuclio.Event(body={'id': '1232'})
handler(context, event)

'"{\\"prediction\\":{\\"17167\\":0.4740774152,\\"566\\":0.2751968181},\\"product_id\\":{\\"17167\\":47209,\\"566\\":22035}}"'

In [309]:
%nuclio deploy -n prediction_server -p recommendation_engine -c

[nuclio.deploy] 2019-04-23 11:29:23,064 (info) Building processor image
[nuclio.deploy] 2019-04-23 11:29:59,475 (info) Pushing image
[nuclio.deploy] 2019-04-23 11:30:06,545 (info) Build complete
[nuclio.deploy] 2019-04-23 11:30:38,892 (warn) Create function failed failed, setting function status
[nuclio.deploy] 2019-04-23 11:30:38,893 
Error - NuclioFunction in error state (
Error - context deadline exceeded
    .../platform/kube/controller/nucliofunction.go:122

Call stack:
Failed to wait for function resources to be available
    .../platform/kube/controller/nucliofunction.go:122
)
    .../nuclio/nuclio/pkg/platform/kube/deployer.go:185

Call stack:
NuclioFunction in error state (
Error - context deadline exceeded
    .../platform/kube/controller/nucliofunction.go:122

Call stack:
Failed to wait for function resources to be available
    .../platform/kube/controller/nucliofunction.go:122
)
    .../nuclio/nuclio/pkg/platform/kube/deployer.go:185
Failed to wait for function readiness.


%nuclio: error: cannot deploy 
